In [1]:
!sudo apt-get update
!sudo apt-get install -y uidmap fuse-overlayfs

# Scarica l'ultima versione di Apptainer (la stessa che avevi scaricato)
!wget -O apptainer.deb https://github.com/apptainer/apptainer/releases/download/v1.2.4/apptainer_1.2.4_amd64.deb

# 1. Tenta di installare il pacchetto (fallirà di nuovo, ma lo registra)
!sudo dpkg -i apptainer.deb

# 2. Usa apt --fix-broken-install per risolvere automaticamente le dipendenze
# e completare la configurazione di Apptainer
!sudo apt --fix-broken install -y

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,143 kB]
Get:8 https://cli.github.com/packages stable/main amd64 Packages [343 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,452 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu j

In [2]:
# Cella Python per la conversione SIF e il download

import subprocess
from google.colab import files

# Definizione delle variabili
# Sostituisci con il tuo GHCR tag corretto in minuscolo
GHCR_IMAGE="ghcr.io/ljpileggi/dockerfile_for_sec_pipeline:latest"
OUTPUT_SIF_NAME="clap_pipeline.sif"

print(f"Esecuzione dell'assemblaggio SIF di {GHCR_IMAGE}...")

# Esegui il pull usando subprocess
# (Usare subprocess in Python è più robusto che %%bash in Colab)
try:
    subprocess.run(
        ["apptainer", "pull", "--force", OUTPUT_SIF_NAME, f"docker://{GHCR_IMAGE}"],
        check=True,
        capture_output=True,
        text=True
    )
    print("Assemblaggio SIF completato con successo.")

except subprocess.CalledProcessError as e:
    print(f"ERRORE CRITICO durante l'assemblaggio SIF:")
    print(e.stderr)
    print("Controlla che il tag dell'immagine sia corretto in GHCR.")

Esecuzione dell'assemblaggio SIF di ghcr.io/ljpileggi/dockerfile_for_sec_pipeline:latest...
Assemblaggio SIF completato con successo.


In [ ]:
# Opzione 1
# Scarica il file SIF
files.download(OUTPUT_SIF_NAME)
print(f"\nIl file '{OUTPUT_SIF_NAME}' è stato scaricato sul tuo PC.")
print("Ora puoi eseguire il trasferimento finale via SCP a CINECA.")

In [3]:
# Opzione 2
# Carica sif su tuo account google drive ed effettua il trasferimento da lì
import subprocess
from google.colab import drive
import os

OUTPUT_SIF_NAME = "clap_pipeline.sif"
DRIVE_DESTINATION_FOLDER = "Cineca_Uploads" # Crea una cartella dedicata su Drive

if not os.path.exists(OUTPUT_SIF_NAME):
    print(f"ERRORE: Il file '{OUTPUT_SIF_NAME}' non è stato trovato.")
else:
    print("Montaggio di Google Drive...")
    # Monta il tuo Google Drive
    drive.mount('/content/drive')

    # Crea la cartella di destinazione su Drive se non esiste
    DRIVE_PATH = f"/content/drive/MyDrive/{DRIVE_DESTINATION_FOLDER}"
    os.makedirs(DRIVE_PATH, exist_ok=True)

    print(f"Caricamento di {OUTPUT_SIF_NAME} su Drive nella cartella: {DRIVE_DESTINATION_FOLDER}")

    # Copia il file nel Drive montato
    subprocess.run(["cp", OUTPUT_SIF_NAME, DRIVE_PATH], check=True)

    print("\nCARICAMENTO SU GOOGLE DRIVE COMPLETATO CON SUCCESSO!")
    print(f"Il file è pronto per essere scaricato da CINECA.")

Montaggio di Google Drive...
Mounted at /content/drive
Caricamento di clap_pipeline.sif su Drive nella cartella: Cineca_Uploads

CARICAMENTO SU GOOGLE DRIVE COMPLETATO CON SUCCESSO!
Il file è pronto per essere scaricato da CINECA.
